In [1]:
import numpy as np
import pandas as pd
import requests
import json
from xml.etree import ElementTree

In [41]:
from time import sleep

In [51]:
# Goodreads API Developer key
key = 'mTBsGSOvfBaN0OF25U5g'
secret = 'AkIOvUnGYksPPicdwWfQVStbr61O0YzzrVu1ZUmpGA'

In [3]:
gr = pd.read_csv('cleaned_goodreads.csv')

In [4]:
gr

,Unnamed: 0,book_id,overall,reviewTime,asin,reviewText
0,0,22551730,4,"Dec 14, 2016",0307408868,Another hard to put down nonfiction book from ...
1,1,18176747,5,"Dec 21, 2016",0062273205,I haven't read many (any?) books that are writ...
2,2,137554,0,"Mar 20, 2014",006073731X,Sacca and Nate recommend
3,3,40955,5,"Dec 21, 2016",0071424911,A truly inspirational book by a truly inspirat...
4,4,9850443,3,"Aug 05, 2012",0062041266,"A fun, dark, slightly comical western about tw..."
...,...,...,...,...,...,...
913006,960443,4405141,3,"Aug 19, 2014",0061698954,While i liked it and appreciated all the infor...
913007,960444,4405141,5,"Apr 15, 2013",0061698954,If you know anyone suffering from an eating di...
913008,960445,4405141,5,"Jul 28, 2015",0061698954,Fabulous insight to what people struggling wit...
913009,960446,4405141,5,"Mar 30, 2009",0061698954,This is an excellent resource -best book I hav...


In [5]:
# Find IDs of unique books to find metadata for
books = np.unique(gr.book_id)
print(len(books))

37276


In [6]:
# Create empty dataframe to store books metadata
df = pd.DataFrame(columns = ['book_id', 'publication_year', 'publication_month', 'publication_day', 'publisher',
                             'description', 'books_count', 'original_publication_year', 'original_publication_month',
                             'original_publication_day', 'media_type', 'num_pages', 'format', 'genres'])

In [7]:
# Add books & genres to dataframe
count = 0
with open('goodreads_book_genres_initial.json', 'r') as f:
    for line in f:
        if count == len(books):
            break
        else:
            j = json.loads(line)
            if int(j['book_id']) in books: 
                df = df.append({'book_id':j['book_id'], 'genres':j['genres']}, ignore_index=True)
                count += 1

In [8]:
df

,book_id,publication_year,publication_month,publication_day,publisher,description,books_count,original_publication_year,original_publication_month,original_publication_day,media_type,num_pages,format,genres
0,780911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'fiction': 2428, 'history, historical fiction..."
1,926667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'fiction': 30, 'poetry': 29}"
2,18498572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'history, historical fiction, biography': 30,..."
3,268464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'fiction': 15, 'history, historical fiction, ..."
4,598199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'history, historical fiction, biography': 10,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37271,5582304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'history, historical fiction, biography': 22,..."
37272,3106983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'comics, graphic': 7696, 'non-fiction': 1811,..."
37273,11873,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'fiction': 688, 'history, historical fiction,..."
37274,823091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'non-fiction': 3}


In [9]:
df.to_csv('genres.csv')

In [10]:
df = pd.read_csv('genres.csv')

In [42]:
# Test 1000 books first
books_subset = books[:1000]

In [43]:
# Crawl for metadata
for book in books_subset:
    # API request
    res = requests.get('https://www.goodreads.com/book/show.xml', params={'key': key, 'id': book})
    # Save HTML into XML
    root = ElementTree.fromstring(res.text) 
    # Save metadata
    publication_year = root.find('book').find('publication_year').text
    publication_month = root.find('book').find('publication_month').text
    publication_day = root.find('book').find('publication_day').text
    publisher = root.find('book').find('publisher').text
    description = root.find('book').find('description').text
    books_count = root.find('book').find('work').find('books_count').text
    original_publication_year = root.find('book').find('work').find('original_publication_year').text
    original_publication_month = root.find('book').find('work').find('original_publication_month').text
    original_publication_day = root.find('book').find('work').find('original_publication_day').text
    media_type = root.find('book').find('work').find('media_type').text
    num_pages = root.find('book').find('num_pages').text
    form = root.find('book').find('format').text
    # Add metadata to dataframe
    df.loc[df.book_id == book, 'publication_year'] = publication_year
    df.loc[df.book_id == book, 'publication_month'] = publication_month
    df.loc[df.book_id == book, 'publication_day'] = publication_day
    df.loc[df.book_id == book, 'publisher'] = publisher
    df.loc[df.book_id == book, 'description'] = description
    df.loc[df.book_id == book, 'books_count'] = books_count
    df.loc[df.book_id == book, 'original_publication_year'] = original_publication_year
    df.loc[df.book_id == book, 'original_publication_month'] = original_publication_month
    df.loc[df.book_id == book, 'original_publication_day'] = original_publication_day
    df.loc[df.book_id == book, 'media_type'] = media_type
    df.loc[df.book_id == book, 'num_pages'] = num_pages
    df.loc[df.book_id == book, 'format'] = form
    sleep(1)

In [44]:
df

,Unnamed: 0,book_id,publication_year,publication_month,publication_day,publisher,description,books_count,original_publication_year,original_publication_month,original_publication_day,media_type,num_pages,format,genres
0,0,780911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'fiction': 2428, 'history, historical fiction..."
1,1,926667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'fiction': 30, 'poetry': 29}"
2,2,18498572,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'history, historical fiction, biography': 30,..."
3,3,268464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'fiction': 15, 'history, historical fiction, ..."
4,4,598199,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'history, historical fiction, biography': 10,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37271,37271,5582304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'history, historical fiction, biography': 22,..."
37272,37272,3106983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'comics, graphic': 7696, 'non-fiction': 1811,..."
37273,37273,11873,2004,7,1,Vintage Classics,WINNER OF THE PULITZER PRIZE<br /><br />In 183...,60,1967,None,None,book,480,Paperback,"{'fiction': 688, 'history, historical fiction,..."
37274,37274,823091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'non-fiction': 3}


In [45]:
df.to_csv('books_metadata.csv')